In [1]:
#import tensorflow as tf
#!pip install tensorflow==2.9.1
import tensorflow as tf
print(tf.__version__)

2.9.1


In [2]:
print(tf.__version__)
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal
#import pywt
import numpy as np
#import imageio
import tensorflow as tf
#import cv2
#import time
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from tensorflow.keras.models import Model, load_model
from sklearn.metrics import classification_report
import os
#from scipy import signal
#from sklearn import preprocessing
print(tf.__version__)
from google.colab import drive
drive.mount('/content/drive')

2.9.1
2.9.1
Mounted at /content/drive


In [21]:
train_data = pd.read_csv('/content/drive/MyDrive/TCC-Physionet/dadosPhysionetTrainBalanceado.csv', delimiter=";")
test_data = pd.read_csv('/content/drive/MyDrive/TCC-Physionet/dadosPhysionetTestBalanceado.csv', delimiter=";")
val_data = pd.read_csv('/content/drive/MyDrive/TCC-Physionet/dadosPhysionetValBalanceado.csv', delimiter=";")


In [22]:
labels = {'0': 'rest', '1': 'left', '2': 'right'}
selected_eletrodes = ['F3', 'F4', 'F5', 'F6', 'FC5', 'FC3',
             'FC4', 'FC6', 'C5', 'C3', 'C4', 'C6'
             ]

In [23]:
def assemble_data(data, selected_eletrodes, std_el, mean_el):

  labels = []
  signals = []
  pacients = data["paciente"].unique()
  len_selected_eletrodes = len(selected_eletrodes)

  for pacient in pacients:
      measures = data.loc[data['paciente'] == pacient]['medicao'].unique()
      for measure in measures:
          label = data.loc[(data['paciente'] == pacient) & (data['medicao'] == measure)]['label'].unique()[0]
          signal_list = []
          img = np.zeros([len_selected_eletrodes,640])
          for idx, eletrode in enumerate(selected_eletrodes):
              sign = data.loc[(data['paciente'] == pacient) & (data['medicao'] == measure) & (data['eletrodo'] == eletrode)].iloc[:,4:].to_numpy().reshape(640)
              sign -= mean_el[idx]
              sign /= std_el[idx] + 1e-7
              img[idx] = sign
          signals.append(img)
          labels.append(label-1)
          '''if label == 1 :
            labels.append([1,0,0])
          elif label == 2:
            labels.append([0,1,0])
          else:
            labels.append([0,0,1])'''

  return labels, signals


In [24]:
def metrics(data, selected_eletrodes):
  std = []
  mean = []
  for eletrode in selected_eletrodes:
    eletrodoStd = pd.Series(data.loc[data["eletrodo"] == eletrode, "leitura0":"leitura639"].values.ravel('F')).std()
    eletrodoMean = pd.Series(data.loc[data["eletrodo"] == eletrode, "leitura0":"leitura639"].values.ravel('F')).mean()
    std.append(eletrodoStd)
    mean.append(eletrodoMean)

  return std, mean

In [25]:
class PhysionetDataset:
  def __init__(self, data, selected_eletrodes, batch_size):
    self.std, self.mean = metrics(data, selected_eletrodes)
    label, signal = assemble_data(data, selected_eletrodes, self.std, self.mean)
    img_generator = tf.keras.preprocessing.image.ImageDataGenerator()

    self.features = img_generator.flow(
        x=np.array(signal).reshape(len(signal), len(selected_eletrodes), 640, 1),
        y=np.array(label).reshape(len(label),1),
        batch_size=batch_size)



In [33]:
batch = 64


train_dataset = PhysionetDataset(train_data, selected_eletrodes, batch)
val_dataset = PhysionetDataset(val_data, selected_eletrodes, batch)
#test_dataset = PhysionetDataset(test_data, selected_eletrodes, batch)

In [19]:
inp = layers.Input(shape=(12, 640, 1))
x = layers.Conv2D(64, (4,4), activation='relu')(inp)
x = layers.MaxPool2D((2,2))(x)
x = layers.BatchNormalization()(x)
x = layers.TimeDistributed(layers.LSTM(256, return_sequences=True))(x)
x = layers.TimeDistributed(layers.LSTM(128, return_sequences=True))(x)
x = layers.TimeDistributed(layers.LSTM(64, return_sequences=True))(x)
x = layers.Flatten()(x)
x = layers.Dense(3, activation='softmax')(x)

model = Model(inp, x)

In [20]:
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 12, 640, 1)]      0         
                                                                 
 conv2d_8 (Conv2D)           (None, 9, 637, 64)        1088      
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 4, 318, 64)       0         
 2D)                                                             
                                                                 
 batch_normalization_2 (Batc  (None, 4, 318, 64)       256       
 hNormalization)                                                 
                                                                 
 time_distributed_24 (TimeDi  (None, 4, 318, 256)      328704    
 stributed)                                                      
                                                           

In [28]:

model.compile(optimizer='adam',loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])

In [30]:
model = load_model('/content/drive/MyDrive/TCC-Physionet/model_CNNLSTM.h5')

history = model.fit(train_dataset.features.x, train_dataset.features.y, epochs=2,
                    validation_data=(val_dataset.features.x, val_dataset.features.y))#, callbacks=callbacks)

model.save('/content/drive/MyDrive/TCC-Physionet/model_CNNLSTM.h5')

Epoch 1/2
50/50 [==============================] - 328s 7s/step - loss: 0.7949 - accuracy: 0.6500 - val_loss: 0.9722 - val_accuracy: 0.5810
Epoch 2/2
50/50 [==============================] - 324s 7s/step - loss: 0.7078 - accuracy: 0.6848 - val_loss: 1.0113 - val_accuracy: 0.5363


In [ ]:
model.save('/content/drive/MyDrive/TCC-Physionet/model_LSTM.h5')

In [31]:
test_dataset = PhysionetDataset(test_data, selected_eletrodes, batch)

In [32]:
score = model.evaluate(test_dataset.features.x, test_dataset.features.y, verbose=0)
print('Loss: ', score[0])
print('Accuracy: ', score[1])

Loss:  0.9526958465576172
Accuracy:  0.5631313323974609
